In [1]:

import numpy as np
from scipy.stats import norm
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import sys
# from ipynb.fs.full.functions import *

In [2]:
sys.path.append('../../')
import function.functions

In [3]:
folder_dir = '../2.array'

list_data = os.listdir(folder_dir)
print(list_data)

['generated_1', 'generated_2', 'generated_3', 'real_1_IBD', 'real_2_DR', 'real_3_HF', 'real_4_TS', 'real_5_HT', 'real_6_BC', 'real_7_MS']


In [4]:
list_data=['real_7_MS']

In [5]:

# list_num = 1


# print(list_data)
# loc = list_data[list_num]
# print(loc)


# Arranging the dataset from best model

## -Features, importance, average rank everything should be arranged by its importance
## -In case of correlation reduction, the importance should be re-caculated and the arrangement should be redone
## -By data cutting and feature cutting, the stability will be compared by the rank in the best model
## -However, in case of generated dataset, the arrangement and comparison should be done by the absolute rank, as we know the answer
## -Which means, the importance in the best model will not be used. Only the performance is required

In [6]:
for loc in list_data:

    print(loc)
    # Data Call
    data_loc = "../2.array/"+loc
    features = np.load(data_loc+'/features.npy')
    label = np.load(data_loc+'/label.npy')


    # Building Directory
    data_loc = "../3.preprocessed/"+loc
    output_loc = "../../result/"+loc

    if not os.path.exists(data_loc):
        os.makedirs(data_loc)
    if not os.path.exists(data_loc+"/corr_cut"):
        os.makedirs(data_loc+"/corr_cut")

    if not os.path.exists(output_loc):
        os.makedirs(output_loc)
    if not os.path.exists(output_loc+"/corr_cut"):
        os.makedirs(output_loc+"/corr_cut")


    # Model Training
    performance,feature_importance,feature_importance_rank = function.functions.random_forest(features,label,output_loc)

    # Arrange by feature importance
    average_importance = np.average(feature_importance,axis=0)
    if loc == 'generated_1':
        average_importance = np.arange(features.shape[1])
    
    arranged_features = features[:,average_importance.argsort()[::-1]].copy()


    np.save(data_loc+'/features.npy',arranged_features)
    np.save(data_loc+'/label.npy',label)

    np.save(output_loc+"/performance.npy",performance)
    np.save(output_loc+"/feature_importance.npy",feature_importance[:,average_importance.argsort()[::-1]])
    np.save(output_loc+"/feature_importance_rank.npy",feature_importance_rank[:,average_importance.argsort()[::-1]])

    if loc == 'generated_1' or loc == 'generated_2' or loc == 'generated_2':
        continue
        
        
    print("Correlation cut start")
    
    for corr_standard in [0.3,0.5,0.7,0.9]:


        features = np.load(data_loc+'/features.npy')
        label = np.load(data_loc+'/label.npy')


        corr_data_loc = data_loc+"/corr_cut/%s"%corr_standard
        corr_output_loc = output_loc +"/corr_cut/%s"%corr_standard 

        if not os.path.exists(corr_data_loc):
            os.makedirs(corr_data_loc)

        if not os.path.exists(corr_output_loc):
            os.makedirs(corr_output_loc)


        finish_work = False
        while True:
            dataframe = pd.DataFrame(features)
            corr_matrix = dataframe.corr(method = 'pearson').abs()
            corr_matrix =  corr_matrix.to_numpy()
            for index_target in range(features.shape[1]):
                
                if index_target==(features.shape[1]-1):
                    finish_work = True
                    break                    
                
                corr_loc = (corr_matrix[index_target,:]>=corr_standard)
                if np.sum(corr_loc)==1:
                    continue
                else:
                    non_corr_loc = (corr_matrix[index_target,:]<corr_standard)
                    non_corr_loc[index_target] =True
                    features = features[:,non_corr_loc]
                    break
            if finish_work:
                break
            
            
            
            

#         # Corr matrix
#         dataframe = pd.DataFrame(features)
# #         dataframe = dataframe.replace(0,np.nan)
#         corr_matrix = dataframe.corr(method = 'pearson').abs()
# #         corr_matrix = corr_matrix.replace(np.nan,0)
#         corr_matrix =  corr_matrix.to_numpy()

#         temp_features = features.copy()
#         temp_corr = corr_matrix.copy()

#         corr_count = 0
#         for i in range(features.shape[1]):
#             if np.sum(temp_corr[i-corr_count,:]>=corr_standard)>1:
#                 temp_features = np.delete(temp_features,i-corr_count,axis=1)
#                 temp_corr = pd.DataFrame(temp_features).corr(method = 'pearson').abs().to_numpy()
#                 corr_count+=1

#         features = temp_features
        
        
        dataframe = pd.DataFrame(features)
        corr_matrix = dataframe.corr(method = 'pearson').abs()
        print("Correlation result: any correlation>standard?")
        print(np.sum(np.sum(corr_matrix>=corr_standard))-features.shape[1])
        print("Number of features left: ")
        print(features.shape[1])



        performance,feature_importance,feature_importance_rank = function.functions.random_forest(features,label,corr_output_loc)

        # Arrange by feature importance
        average_importance = np.average(feature_importance,axis=0)
        arranged_features = features[:,average_importance.argsort()[::-1]].copy()

        np.save(corr_data_loc+'/features.npy',arranged_features)
        np.save(corr_data_loc+'/label.npy',label)

        np.save(corr_output_loc+"/performance.npy",performance)
        np.save(corr_output_loc+"/feature_importance.npy",feature_importance[:,average_importance.argsort()[::-1]])
        np.save(corr_output_loc+"/feature_importance_rank.npy",feature_importance_rank[:,average_importance.argsort()[::-1]])


real_7_MS
Correlation cut start
Correlation result: any correlation>standard?
0
Number of features left: 
11
Correlation result: any correlation>standard?
0
Number of features left: 
17
Correlation result: any correlation>standard?
0
Number of features left: 
19
Correlation result: any correlation>standard?
0
Number of features left: 
19


In [7]:
features.shape

(2009, 19)

In [2]:
17/21

0.8095238095238095

<!-- # Generation from real data -->

<!-- ## -Feature importance calculated by real dataset are used for coefficient 
## -As we know the coefficient, the features don't need to be re-arranged, just following arranged real dataset
## -The comparison by cutting will be done by absolute rank, top from the first -->

In [51]:
# for loc in list_data:
#     if loc == 'generated_1':
#         continue

#     print(loc)
    
#     data_loc = "../3.preprocessed/"+loc
#     output_loc = "../../result/"+loc

#     features = np.load(data_loc+'/features.npy')
#     label = np.load(data_loc+'/label.npy')

#     arranged_importance = np.load(output_loc+"/feature_importance.npy")
    
#     arranged_importance = np.average(arranged_importance,axis=0)
    

#     # new loc
#     loc_generate = "generated_"+loc
#     data_loc = "../3.preprocessed/"+loc_generate
#     output_loc = "../../result/"+loc_generate

#     if not os.path.exists(data_loc):
#         os.makedirs(data_loc)

#     if not os.path.exists(data_loc+"/corr_cut"):
#         os.makedirs(data_loc+"/corr_cut")

#     if not os.path.exists(output_loc):
#         os.makedirs(output_loc)
#     if not os.path.exists(output_loc+"/corr_cut"):
#         os.makedirs(output_loc+"/corr_cut")



#     generated_label = np.zeros((label.shape[0],))
#     for index in range(label.shape[0]):
#         generated_label[index] = np.sum(np.multiply(features[index,:],arranged_importance))




#     binary_generated_label =np.zeros((generated_label.shape[0],))
#     print("Original number of positive")
#     print(int(np.sum(label)))
#     standard = generated_label[generated_label.argsort()[::-1]][int(np.sum(label))]
#     loc_up = (generated_label>standard)
#     loc_down = (generated_label<=standard)
#     binary_generated_label[loc_up]  =1
#     binary_generated_label[loc_down] =0


#     label = binary_generated_label

#     print("Generated number of positive")
#     print(int(np.sum(label)))

#     performance,feature_importance,feature_importance_rank = function.functions.random_forest(features,label,output_loc)

#     np.save(data_loc+'/features.npy',features)
#     np.save(data_loc+'/label.npy',label)


#     # 원래의 arranged된 importance를 가져와서 저장
#     output_loc = "../../result/"+loc
#     arranged_importance = np.load(output_loc+"/feature_importance.npy")
#     arranged_importance_rank = np.load(output_loc+"/feature_importance_rank.npy")

#     output_loc = "../../result/"+loc_generate

#     np.save(output_loc+"/performance.npy",performance)
#     np.save(output_loc+"/feature_importance.npy",arranged_importance)
#     np.save(output_loc+"/feature_importance_rank.npy",arranged_importance_rank)


    


#     for corr_standard in [0.3,0.5,0.7,0.9]:

#         corr_data_loc = data_loc+"/corr_cut/%s"%corr_standard
#         corr_output_loc = output_loc +"/corr_cut/%s"%corr_standard 

#         if not os.path.exists(corr_data_loc):
#             os.makedirs(corr_data_loc)
#         if not os.path.exists(corr_output_loc):
#             os.makedirs(corr_output_loc)


            
            
        
#         # corr cut 된, arranged 된 original data를 가져옴
            
#         org_data_loc = "../3.preprocessed/"+loc+"/corr_cut/%s"%corr_standard
#         org_output_loc = "../../result/"+loc+"/corr_cut/%s"%corr_standard

#         features = np.load(org_data_loc+'/features.npy')
#         label = np.load(org_data_loc+'/label.npy')
#         feature_importance = np.load(org_output_loc+"/feature_importance.npy")
#         arranged_importance = np.average(feature_importance,axis=0)
            
            
        

#         generated_label = np.zeros((label.shape[0],))
#         for index in range(label.shape[0]):
#             generated_label[index] = np.sum(np.multiply(features[index,:],arranged_importance))


#         binary_generated_label =np.zeros((generated_label.shape[0],))
#         print("Original number of positive")
#         print(int(np.sum(label)))
#         standard = generated_label[generated_label.argsort()[::-1]][int(np.sum(label))]
#         loc_up = (generated_label>standard)
#         loc_down = (generated_label<=standard)
#         binary_generated_label[loc_up]  =1
#         binary_generated_label[loc_down] =0

#         label = binary_generated_label

            
#         performance,feature_importance,feature_importance_rank = function.functions.random_forest(features,label,corr_output_loc)


        


#         np.save(corr_data_loc+'/features.npy',features)
#         np.save(corr_data_loc+'/label.npy',label)

#         np.save(corr_output_loc+"/performance.npy",performance)
        
        
#         # 원래의 arranged된 importance를 가져와서 저장
#         feature_importance = np.load(org_output_loc+"/feature_importance.npy")
#         feature_importance_rank = np.load(org_output_loc+"/feature_importance_rank.npy")
#         np.save(corr_output_loc+"/feature_importance.npy",feature_importance)
#         np.save(corr_output_loc+"/feature_importance_rank.npy",feature_importance_rank)


real_1_IBD
Original number of positive
702
Generated number of positive
702
Original number of positive
702
Original number of positive
702
Original number of positive
702
Original number of positive
702
real_2_DR
Original number of positive
611
Generated number of positive
611
Original number of positive
611
Original number of positive
611
Original number of positive
611
Original number of positive
611
real_3_HF
Original number of positive
96
Generated number of positive
96
Original number of positive
96
Original number of positive
96
Original number of positive
96
Original number of positive
96
real_4_PK
Original number of positive
147
Generated number of positive
147
Original number of positive
147
Original number of positive
147
Original number of positive
147
Original number of positive
147
real_5_TS
Original number of positive
70
Generated number of positive
70
Original number of positive
70
Original number of positive
70
Original number of positive
70
Original number of positive